## GPT как персональный раб разработчика

Используем большие языковые модели для автоматизации сложных задач.

Для начала научимся использовать большие языковые модели программно. Я рекомендую посмотреть на библиотеку [LangChain](https://www.langchain.com/).

> Если вы открыли код из Google Colab, вам нужно создать файл `config.json`, содержащий ключи для доступа к моделям, следующего вида:
```json
{
    "folder_id" : "...",
    "api_key" : "...",
    "gigachain_auth" : "..."
}
```

Для поддержки модели Yandex GPT можно дополнительно установить библиотеку [`yandex_chain`](https://github.com/yandex-datasphere/yandex-chain), в которой чуть больше возможностей по работе с YandexGPT, чем в стандартной LangChain.

Для начала, установим библиотеки:

In [6]:
%pip install yandex_chain==0.0.7 gigachat
%pip install --upgrade yandexcloud
%pip install g4f

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 753.7 kB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dvc 2.58.2 requires platformdirs<4,>=3.1.1, but you have platformdirs 4.2.0 which is incompatible.
dvc 2.58.2 requires psutil>=5.8, but you have psutil 5.7.3 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 791.3 kB/s eta 0:00

> Возможно, для корректной работы кода ниже придётся перезапустить Kernel ноутбука

Вот как просто можно организовать вызов языковой модели Yandex GPT из кода:

In [1]:
from langchain_community.llms import YandexGPT, GigaChat
from yandex_chain import YandexLLM, YandexGPTModel
import json

config = json.load(open('config.json'))

YGPT = YandexLLM(
    folder_id = config['folder_id'], 
    api_key = config['api_key'], 
    model = YandexGPTModel.Pro,
    temperature=0.01,)
GC = GigaChat(credentials=config['gigachain_auth'],verify_ssl_certs=False)

print(YGPT.invoke("Напиши сказку про мальчика, который любил JSON"))

В далёкой стране, где магия и технологии сосуществовали в гармонии, жил мальчик по имени Джек. Он был очень любознательным и всегда стремился к новым знаниям. Джек любил изучать различные языки программирования и особенно выделял JSON.

Однажды Джек решил создать свой собственный мир, где всё будет основано на JSON. Он начал с создания простых объектов и массивов, которые представляли собой различные элементы его мира. Джек назвал свой мир «JSONia» и населил его разнообразными существами, каждое из которых имело свой уникальный JSON-объект.

В JSONia Джек создал множество интересных мест, таких как горы, реки, леса и города. Каждое место имело свой уникальный JSON-объект, который определял его характеристики. Например, горы были представлены массивом с высотой, шириной и длиной, а реки — объектом с направлением и скоростью течения.

Джек также создал в JSONia множество существ, каждое из которых имело свой уникальный JSON-объект. Например, он создал драконов, которые были представлены 

Для вызова ChatGPT попробуем использовать библиотеку GPT4Free:

In [3]:
import g4f 

def GPT4(x):
    response = g4f.ChatCompletion.create(
    model=g4f.models.default,
    messages=[{"role": "user", "content": x }])
    return response

res = GPT4('Придумай сказку про мальчика, который любил JSON')
print(res)

Теперь попробуем сделать что-то полезное:

In [5]:
res = YGPT.invoke("Придумай 10 смешных кличек для собаки программиста")
print(res)

Вот несколько забавных кличек для собаки программиста:
1. Байбайнер.
2. Кодер.
3. Хакер.
4. Айтишка.
5. Компилятор.
6. Хакерша.
7. Цифропес.
8. Скриптоша.
9. Алгоритмик.
10. Программинка.

Эти клички могут развеселить и поднять настроение. Однако выбор клички для собаки — это индивидуальный процесс, который зависит от личных предпочтений и особенностей питомца.


## Основные приёмы промптинга

Важно, чтобы модель получила чёткие и понятные инструкции по тому, что же ей нужно сделать.

#### Используем ограничители

In [7]:
text = """
Вы должны выразить то, что вы хотите, чтобы модель сделала, 
предоставив инструкции, которые максимально ясны и конкретны.
Это направит модель на желаемый результат и уменьшит вероятность
получения несвязанных или неправильных ответов. Не путайте
написание четкого запроса с написанием короткого запроса. 
Во многих случаях более длинные запросы обеспечивают большую ясность 
и контекст для модели, что может привести к более подробным 
и соответствующим ответам.
"""

instr = """
    Сократи текст, выделенный тройными обратными
    кавычками, до одного предложения. Выведи в качестве результата
    одно предложение, содержащее главную мысль текста.
    ```{}```"""

res = YGPT.invoke(instr.format(text))
print(res)

Чтобы получить от модели желаемый результат, нужно предоставить ей чёткие и конкретные инструкции, при этом запрос не обязательно должен быть коротким.


Для работы с шаблонами промптов в LangChain есть специальные классы

In [8]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""
    Сократи текст, выделенный тройными обратными
    кавычками, до одного предложения. Выведи в качестве результата
    одно предложение, содержащее главную мысль текста.
    ```{text}```""",
    input_variables=["text"],
)

res = YGPT.invoke(prompt.format(text=text))
print(res)

Чтобы получить от модели желаемый результат, нужно предоставить ей чёткие и конкретные инструкции, при этом запрос не обязательно должен быть коротким.


#### Используем структурированный вывод

In [10]:
YGPT.temperature=0.5
res = YGPT.invoke("Придумай 10 смешных кличек для собаки программиста и выведи результат в формате JSON")
print(res)

Вот десять смешных кличек для собаки программиста, которые я могу предложить:

```json
[
  "КодОбормот",
  "Цифропёс",
  "Байт-Байт",
  "Бит-Бит",
  "Хакер-Бутявка",
  "Компилятор",
  "Флешка",
  "Сбой",
  "Софт-Пёс",
  "Клавиша"
]
```

Эти клички могут быть забавными и подходящими для собаки программиста. Однако выбор клички для собаки зависит от многих факторов, таких как порода, характер и личные предпочтения владельца.


Чтобы убедиться в том, что вывод соответствуюет некоторому формату, используют выходные парсеры:

In [11]:
from langchain.output_parsers import CommaSeparatedListOutputParser

csv_parser = CommaSeparatedListOutputParser()

YGPT.temperature = 0.01

prompt = PromptTemplate(
    template="Придумай 10 смешных {subject}, которые бы были оригинальными. {format_instructions}. Используй формат CSV в одну строку",
    input_variables=["subject"],
    output_parser=csv_parser,
    partial_variables={ "format_instructions" : csv_parser.get_format_instructions() }
)
res = YGPT.invoke(prompt.format(subject="кличек для собаки программиста"))
print(csv_parser.parse(res))

['1. Компилятор', '2. Баг', '3. Алгоритм', '4. Бит', '5. Хакер', '6. Код', '7. Клариса', '8. Цифра', '9. Скрипт', '10. Монитор.']


Поэкспериментируем с температурой

In [13]:
import time 

for t in [0.1, 0.5, 0.9]:
    YGPT.temperature = t
    res = YGPT.invoke(prompt.format(subject="кличек для собаки программиста"))
    time.sleep(1)
    print(f"{t} -> {csv_parser.parse(res)}")

YGPT.temperature = 0.2

0.1 -> ['1. Компилятор', '2. Баг', '3. Оверфлоу', '4. Хакер', '5. Алгоритм', '6. Бит', '7. Байт', '8. Вирус', '9. Код', '10. Сервер.']
0.5 -> ['Компилятор', 'Баг', 'Хакер', 'Ошибочка', 'Алгоритмик', 'Кодзима', 'Глюк', 'Скрипто', 'Байбай', 'Цифра.']
0.9 -> ['УмныйПсст', 'Байбай', 'ТурбоПаскаль', 'МатрицаЛабрадоров', 'Цапцарапбайт', 'Байтерьер', 'Поскребкин', 'Хакергав', 'Алгоритмик', 'Компик.']


#### Используем условия

Попробуем использовать GPT для выделения последовательности инструкций из текста:

In [42]:
text1 = """
Чтобы приготовить омлет, сначала надо взять яйца. Разбиваем их молотком, затем
аккуратно извлекаем осколки скорлупы. Затем добавляем соли. В конце кладем масло на
сковородку, и выливаем туда яичную смесь.
"""

text2 = """
Яичный омлет - это прекрасный завтрак! Вам обязательно стоит его попробовать, если
раньше никогда не пробовали!
"""

prompt = PromptTemplate(
    template="""
    Тебе будет дан текст, выделенный тройными обратными кавычками, в котором содержится
    последовательность инструкций. Перепиши их
    в виде последовательных шагов в таком формате:
    Шаг 1 - ...
    Шаг 2 - ...
    ...
    Шаг N - ... 
    ```{text}```""",
    input_variables=["text"],
)

res = GC.invoke(prompt.format(text=text2))
print(res)

Шаг 1 - Попробуйте яичный омлет на завтрак;
Шаг 2 - Если раньше не пробовали, то обязательно стоит попробовать;
Шаг 3 - Наслаждайтесь вкусом и ароматом этого блюда.


Наблюдаете галлюцинации, при попытке выделить инструции из `text2`?

Чтобы этого избежать, слегка модифицируем инструкцию:

In [41]:
prompt = PromptTemplate(
    template="""
    Тебе будет дан текст, выделенный тройными обратными кавычками.
    Если в тексте нет конкретных инструкций по приготовлению блюда, напиши "Инструкций нет".Если
    в тексте содержится последовательность инструкций, перепиши их
    в виде последовательных шагов в таком формате:
    Шаг 1 - ...
    Шаг 2 - ...
    ...
    Шаг N - ... 
    ```{text}```""",
    input_variables=["text"],
)

res = GC.invoke(prompt.format(text=text2))
print(res)

Инструкций нет


#### Few-Shot

Few-Shot Learning - это когда мы пытаемся "научить" модель прямо в запросе, дав ей несколько примеров. Это может быть полезно как для задания точного формата вывода, так и для формулирования самого задания, например:

In [43]:
YGPT.model = YandexGPTModel.Pro
YGPT.temperature = 0.1
res = YGPT.invoke("""
Пожалуйста, ответь на вопрос ребенка в похожем стиле, продолжив диалог:
    
[Ребенок]: Расскажи мне о терпеливости.
[Родитель]: Терпеливость - это как бесконечная река, которая 
течет сквозь равнины, и никогда не заканчивается. Этой реке
никогда не надоедает течь, потому что она всегда спокойна и
умиротворена.
    
[Ребенок]: Расскажи мне об искренности.
[Родитель]:
""")

print(res)

Искренность — это как яркое солнце, которое светит всем одинаково. Оно не выбирает, кому отдавать своё тепло, и не скрывает своего света. Искренность — это когда ты говоришь правду и не пытаешься обмануть других людей.


## Дайте модели время подумать!

Языковые модели не могут рассуждать, как человек, гоняя мысли в голове "взад-вперёд". Модель всегда генерирует текст "вперёд", и "рассуждает" в процессе генерации. Поэтому важно инструктировать модель так, чтобы она могла "рассуждать вслух".

In [44]:
text = """
Использовать генеративный ИИ полезно, потому что это очень 
сильно ускоряет работу. Также, работая с ChatGPT, мы можем
многому у него научиться. Используя передовые технологии,
мы будем современными и не отставать от прогресса. Но есть риск,
что мы при этом разучимся сами писать.
"""

prompt = PromptTemplate(
    template="""
    Тебе нужно сделать следующее:
    1. Выдели умные мысли, которые содержатся в тексте ниже, 
    выделенном тройными обратными кавычками.
    2. Построй список из всех умных мыслей
    2. Для каждой умной мысли определи, является ли она позитивной
    или негативной.
    3. Выведи ответ в формате JSON, который содержит список
    умных мыслей и их позитивность/негативность.
    ```{text}```""",
    input_variables=["text"],
)

res = YGPT.invoke(prompt.format(text=text))
print(res)

**Умные мысли из текста:**

1. «Использовать генеративный ИИ полезно, потому что это очень сильно ускоряет работу».

2. «Работая с ChatGPT, мы можем многому у него научиться».

3. «Используя передовые технологии, мы будем современными и не отставать от прогресса».

4. «Есть риск, что мы при этом разучимся сами писать».

**Позитивные мысли:**

1. «Использовать генеративный ИИ полезно, потому что это очень сильно ускоряет работу».

2. «Используя передовые технологии, мы будем современными и не отставать от прогресса».

**Негативные мысли:**

1. «Есть риск, что мы при этом разучимся сами писать».


In [46]:
text = """
Использовать генеративный ИИ полезно, потому что это очень 
сильно ускоряет работу. Также, работая с ChatGPT, мы можем
многому у него научиться. Но есть риск,
что мы при этом разучимся сами писать. Используя передовые технологии,
мы будем современными и не отставать от прогресса. 
"""

prompt = PromptTemplate(
    template="""
    Тебе нужно сделать следующее:
    1. Выдели умные мысли, которые содержатся в тексте ниже, 
    выделенном тройными обратными кавычками.
    2. Построй список из всех умных мыслей
    2. Для каждой умной мысли определи, является ли она позитивной
    или негативной.
    3. Выведи ответ в формате JSON, который содержит список
    умных мыслей и их позитивность/негативность.
    Используй следующий формат:
    Текст: <исходный текст с мыслями>
    Умные мысли: <список умных мыслей>
    Позитивные мысли: <список позитивных мыслей>
    Негативные мысли: <список негативных мыслей>
    
    Вот текст, с которым тебе надо работать:
    ```{text}```""",
    input_variables=["text"],
)

YGPT.temperature=0.01
res = YGPT.invoke(prompt.format(text=text))
print(res)

**Текст**: Использовать генеративный ИИ полезно, потому что это очень сильно ускоряет работу. Также, работая с ChatGPT, мы можем многому у него научиться. Но есть риск, что мы при этом разучимся сами писать. Используя передовые технологии, мы будем современными и не отставать от прогресса.

**Умные мысли**:
* Использовать генеративный ИИ полезно, потому что это очень сильно ускоряет работу.
* Работая с ChatGPT, мы можем многому у него научиться.
* Используя передовые технологии, мы будем современными и не отставать от прогресса.

**Позитивные мысли**:
* Использовать генеративный ИИ полезно, потому что это очень сильно ускоряет работу.
* Работая с ChatGPT, мы можем многому у него научиться.
* Используя передовые технологии, мы будем современными и не отставать от прогресса.

**Негативные мысли**:
* Есть риск, что мы при этом разучимся сами писать.

**Ответ**:
```
Текст: Использовать генеративный ИИ полезно, потому что это очень сильно ускоряет работу. Также, работая с ChatGPT, мы можем 

#### Проверка решения

In [49]:
template = """
Тебе необходимо проверить решение задачи по математике студентом. Напиши, правильное
ли решение студента или нет.

Задача:
Необходимо посчитать стоимость уборки в доме площадью 20 кв.метров. 
Стоимость уборки складывается из:
- приезд уборщика - 200 руб.
- мытьё полов - 100 руб. за кв. метр.
- уборка кухни - 500 руб.
- чистка полов - 50 руб. за кв. метр.

Решение студента:
{solution}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["solution"],
)

correct = """
Стоимость уборки:
- приезд уборщика - 200 руб.
- мытьё полов - 100 руб. * 20 кв. метров = 2000 руб.
- уборка кухни - 500 руб.
- чистка полов - 50 руб. * 20 кв. метров = 1000 руб.
Общая стоимость: 200 руб. + 2000 руб. + 500 руб. + 1000 руб. = 3700 руб.
Ответ: 3700 руб.
"""

incorrect = """
Стоимость уборки:
- приезд уборщика - 200 руб.
- мытьё полов - 100 руб. * 20 кв. метров = 2000 руб.
- уборка кухни - 500 руб.
- чистка полов - 5 руб. * 20 кв. метров = 100 руб.
Общая стоимость: 200 руб. + 2000 руб. + 500 руб. + 100 руб. = 2800 руб.
Ответ: 2800 руб.
"""

print(YGPT(prompt.format(solution=incorrect)))

Решение студента правильное. Ответ: 2800 руб.


In [50]:
res = YGPT.invoke("""
Пожалуйста, реши по шагам следующую задачу:
Необходимо посчитать стоимость уборки в доме площадью 20 кв.метров. 
Стоимость уборки складывается из:
- приезд уборщика - 200 руб.
- мытьё полов - 100 руб. за кв. метр.
- уборка кухни - 500 руб.
- чистка полов - 50 руб. за кв. метр.
""")
print(res)

**Шаг 1.**

Необходимо вычислить площадь дома, которая подлежит уборке. По условию задачи, площадь дома составляет 20 кв. метров.

**Шаг 2.**

Стоимость уборки складывается из нескольких составляющих:

* приезд уборщика — 200 руб.;
* мытьё полов — 100 руб. за кв. метр;
* уборка кухни — 500 руб.;
* чистка полов — 50 руб. за кв. метр.

**Шаг 3.**

Рассчитаем стоимость мытья полов:

* 100 * 20 = 2000 руб.

**Шаг 4.**

Рассчитаем стоимость чистки полов:

* 50 * 20 = 1000 руб.

**Шаг 5.**

Сложим все составляющие:

* 200 + 2000 + 500 + 1000 = 3900 руб.

**Ответ:** стоимость уборки в доме площадью 20 кв. метров составит 3900 руб.


In [52]:
template = """
Тебе необходимо проверить решение задачи по математике студентом, которое приведено
ниже в тройных обратных кавычках. Напиши, правильное
ли решение студента или нет. Тебе необходимо сделать следующее:
1. Сначала, реши задачу самостоятельно и выведи пошаговое решение.
2. Сравни решение студента с твоим решением и скажи, правильно ли
решение студента.
Не принимай решения о том, правильно ли студент решил задачу, пока не 
решишь её самостоятельно.
В качестве ответа представь своё решение и напиши, правильно ли студент
решил задачу, и где он ошибся.

Задача:
Необходимо посчитать стоимость уборки в доме площадью 20 кв.метров. 
Стоимость уборки складывается из:
- приезд уборщика - 200 руб.
- мытьё полов - 100 руб. за кв. метр.
- уборка кухни - 500 руб.
- чистка полов - 50 руб. за кв. метр.

Решение студента:
```{solution}```
Напоминаю, что тебе нужно самой решить задачу, и потом сравнить своё решение с решением студента.
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["solution"],
)

correct = """
Стоимость уборки:
- приезд уборщика - 200 руб.
- мытьё полов - 100 руб. * 20 кв. метров = 2000 руб.
- уборка кухни - 500 руб.
- чистка полов - 50 руб. * 20 кв. метров = 1000 руб.
Общая стоимость: 200 руб. + 2000 руб. + 500 руб. + 1000 руб. = 3700 руб.
Ответ: 3700 руб.
"""

incorrect = """
Стоимость уборки:
- приезд уборщика - 200 руб.
- мытьё полов - 100 руб. * 20 кв. метров = 2000 руб.
- уборка кухни - 500 руб.
- чистка полов - 5 руб. * 20 кв. метров = 100 руб.
Общая стоимость: 200 руб. + 2000 руб. + 500 руб. + 100 руб. = 2800 руб.
Ответ: 2800 руб.
"""

print(YGPT.invoke(prompt.format(solution=correct)))

**Решение задачи:**

1. Приезд уборщика — 200 руб.

2. Мытьё полов — 100 руб. * 20 кв. метров = 2000 руб.

3. Уборка кухни — 500 руб.

4. Чистка полов — 50 руб. * 20 кв. метров = 1000 руб.

Общая стоимость: 200 руб. + 2000 руб. + 500 руб. + 1000 руб. = 3700 руб.

**Ответ: 3700 руб.**

Решение студента правильное.


## Итеративная разработка промптов

Очень важный момент в промпт-инжиниринге - почти никогда хороший результат не получается с первого раза! Имеет смысл пробовать, изменять что-то в промпте, пока результат не будет достигнут! Также помогает экспериментировать с температурой.

In [55]:
techspec = """
Название: Xiaomi Mi 9
Процессор: SnapDragon 855
Зарядка: 20Вт беспроводная
Дисплей: Samsung AMOLED, 6.39 дюймов
Фото: 48 Мп SONY
Фронтальная камера: 20 Мп
Объективы: 3 шт., 177 град. широкоугольный
Стекло: Corning Gorilla Glass 6
ОС: MIUI 10
Память: 6Гб + 128 Гб
Разрешение: 2340 x 1080 FHD+ 403 PPI
Яркость: 600 нит (HBM) / 430 нит (тип)
"""

template = """"
Ты должен помочь отделу маркетинга сформировать привлекательное описание
модели сотового телефона для потребителя. Описание приводится
ниже в тройных обратных кавычках:
```{techspec}```
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["techspec"],
)

YGPT.temperature = 0.4
res = YGPT.invoke(prompt.format(techspec=techspec))
print(res)

Встречайте **Xiaomi Mi 9** — мощный смартфон с передовыми характеристиками, который станет вашим верным спутником в мире технологий!

**Xiaomi Mi 9** оснащён мощным процессором **SnapDragon 855**, который обеспечивает быструю и плавную работу всех приложений. Благодаря беспроводной зарядке мощностью **20 Вт** вы сможете быстро и удобно заряжать свой смартфон.

Большой **AMOLED-дисплей Samsung** с диагональю **6.39 дюймов** и разрешением **2340 x 1080 FHD+** (403 PPI) подарит вам яркое и чёткое изображение. А благодаря яркости в **600 нит (HBM) / 430 нит (тип)** вы сможете наслаждаться просмотром видео и фотографий даже при ярком солнечном свете.

Камера **Xiaomi Mi 9** не оставит вас равнодушными. С её помощью вы сможете делать потрясающие снимки благодаря **48 Мп SONY** и 3 объективам с углом обзора в **177 градусов**. А фронтальная камера на **20 Мп** сделает ваши селфи идеальными.

**Стекло Corning Gorilla Glass 6** надёжно защитит ваш смартфон от повреждений. А операционная система

Регулируем длину и целевую аудиторию:

In [57]:
template = """"
Ты должен помочь отделу маркетинга сформировать подробное 
привлекательное описание модели сотового телефона для потребителя, состоящее
из двух абзацев текста.
Необходимо сосредоточиться на его преимуществах для фотографов, которые
любят путешествовать. Описание приводится
ниже в тройных обратных кавычках:
```{techspec}```
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["techspec"],
)

YGPT.temperature = 0.4
res = YGPT.invoke(prompt.format(techspec=techspec))
print(res)

**Xiaomi Mi 9** – это мощный смартфон, который станет незаменимым помощником для фотографов, любящих путешествовать.

С этим смартфоном вы сможете делать потрясающие снимки даже в самых экстремальных условиях. Широкоугольный объектив с углом обзора 177 градусов позволит вам снимать захватывающие пейзажи, а высокое разрешение 48 Мп и фронтальная камера 20 Мп обеспечат чёткость и яркость ваших фотографий.

Xiaomi Mi 9 оснащён большим AMOLED-дисплеем с высокой яркостью и прочным стеклом Corning Gorilla Glass 6, что делает его надёжным спутником в любых путешествиях. А благодаря быстрой беспроводной зарядке мощностью 20 Вт и операционной системе MIUI 10, вы сможете быстро и удобно использовать все функции смартфона.


## Основные приёмы использования 

1. Генерация текста по данным (экспансия)
2. Извлечение данных из текста (экстракция)
3. Суммаризация текста
4. Десуммаризация текста
5. Переписывание текста (тональность, акцент)
6. Преобразование текста (перевод)

### Пример

Рассмотрим пример суммаризации множества отзывов, чтобы можно было охватить их одним взглядом:

In [59]:
import time
reviews = ["""
Я посетил ресторан Макдональдс летом прошлого года, и был разочарован!
Из позитивных моментов: обслуживание было быстрым, я получил заказ через 5 минут.
Но при этом весь персонал был мрачным, и еда оказалась не очень вкусной. Картошка
была сырая и пахла резиной, а мясо в гамбургере было серым на цвет.
""","""
Я слышал, что в Макдональдсе котлеты готовят не из мяса, и 
сегодня я в этом убедился сам! В котлете попалось что-то жесткое,
и я чуть не сломал зуб!
""","""
Я был а Макдональдсе четыре раза, и каждый раз это было удивительно!
Столько вкусов мороженого я никогда не пробовал! И все официантки за
кассой очень молодые и симпатичные!
""","""
Макдональдс - это прекрасное место, где можно поесть американскую еду:
гамбургеры, картошку фри и конечно же прекрасное мороженое!
Я обычно заказываю биг мак, в котором много вкусного зелёного салата.
Это делает еду полезной и здоровой, что очень хорошо! Спасибо всем официантам,
которые всегда улыбаются и радуются мне!
"""]

template = """"
Ниже в тройных обратных кавычках приводится отзыв посетитея о ресторане. Пожалуйста,
перефразируй отзыв коротко в одном предложении, от третьего лица:
```{review}```
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["review"],
)

YGPT.temperature = 0.01
for r in reviews:
    res = YGPT.invoke(prompt.format(review=r))
    print(res)
    time.sleep(1)

Посетитель ресторана Макдональдс прошлым летом остался разочарован: обслуживание было быстрым, но весь персонал был мрачным, а еда — невкусной.
Посетитель утверждает, что в ресторане ему попалась жёсткая котлета, и он чуть не сломал зуб.
Посетителю очень понравилось в «Макдоналдсе»: он отметил разнообразие вкусов мороженого и работу симпатичных молодых официанток.
В ресторане «Макдоналдс» можно попробовать американскую еду, например, биг-мак с зелёным салатом, а также получить позитивный заряд от приветливых официантов.


Можно также сконцентрировать отзывы на каком-то одном интересующем нас аспекте:

In [60]:
template = """"
Ниже в тройных обратных кавычках приводится отзыв посетитея о ресторане. Пожалуйста,
перефразируй отзыв коротко в одном предложении, обратив внимание исключительно на
качество еды:
```{review}```
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["review"],
)

YGPT.temperature = 0.01
for r in reviews:
    res = YGPT.invoke(prompt.format(review=r))
    print(res)
    time.sleep(1)

Летом прошлого года я посетил ресторан Макдональдс, остался разочарован обслуживанием и качеством еды, особенно меня расстроила сырая картошка с запахом резины и невкусное мясо в гамбургере.
В ресторане посетителю попалась жёсткая котлета, и он чуть не сломал зуб.
В Макдональдсе подают удивительно вкусное мороженое.
В «Макдоналдс» можно попробовать вкусную и здоровую американскую еду, например, биг-мак с зелёным салатом.


А если на нужен более подробный анализ отзывов - мы можем прибегнуть к извлечению данных в формате JSON, для последующего анализа:

In [61]:
template = '''
Ниже в тройных обратных кавычках приводится отзыв посетитея о ресторане. Пожалуйста,
прочитай этот отзыв, и извлеки из него следующую информацию:
1. Качество обслуживания
2. Качесвто еды
3. Общая тональность отзыва: положительный или отрицательный.
Результат верни в формате JSON такого вида:
{{
  "обслуживание" : "...",
  "еда" : "...",
  "тональность" : "..."
}}
Вот сам отзыв:
```{review}```
'''

prompt = PromptTemplate(
    template=template,
    input_variables=["review"],
)

YGPT.temperature = 0.5
for r in reviews:
    res = YGPT.invoke(prompt.format(review=r))
    print(res)
    time.sleep(1)

{
  "обслуживание": "быстрое",
  "еда": "невкусная",
  "тональность": "отрицательный"
}
{
  "обслуживание": "",
  "еда": "жёсткая",
  "тональность": "отрицательный"
}
{
  "обслуживание": "удивительно",
  "еда": "столько вкусов мороженого я никогда не пробовал",
  "тональность": "положительный"
}
{
  "обслуживание": "хорошее",
  "еда": "вкусная",
  "тональность": "положительная"
}


Для более точного парсинга стоит использовать `JsonOutputParser`. Заодно попросим извлечь побольше разной информации в одном запросе:

In [62]:
from langchain.output_parsers.json import SimpleJsonOutputParser

parser = SimpleJsonOutputParser()

template = '''
Ниже в тройных обратных кавычках приводится отзыв посетитея о ресторане. Пожалуйста,
прочитай этот отзыв, и извлеки из него следующую информацию:
1. Качество обслуживания (оцени в диапазоне от 1 - плохо, до 5 - отлично)
2. Качество еды (оцени в диапазоне от 1 - плохо, до 5 - отлично)
3. Общая тональность отзыва: положительный (1) или отрицательный (-1).
4. Краткое содержание отзыва в 5-7 словах.
5. Перевод краткого отзыва на английский.
Результат верни в формате JSON такого вида:
{{
  "обслуживание" : <оценка>,
  "еда" : <оценка>,
  "тональность" : <оценка>,
  "саммари" : "<краткое содержание>",
  "англ" : "<перевод краткого отзыва на английский>"
}}
Вот сам отзыв:
```{review}```
'''

prompt = PromptTemplate(
    template=template,
    input_variables=["review"],
)

tab = []
YGPT.temperature = 0.5
for r in reviews:
    res = YGPT.invoke(prompt.format(review=r))
    js = parser.parse(res)
    print(js)
    tab.append(js)
    time.sleep(1)

import pandas as pd
pd.DataFrame(tab)

{'обслуживание': 4, 'еда': 1, 'тональность': -1, 'саммари': 'Разочарование в обслуживании и еде', 'англ': 'I was disappointed with the service and food.'}
{'обслуживание': 1, 'еда': 1, 'тональность': -1, 'саммари': 'Некачественная еда', 'англ': 'Poor food quality'}
{'обслуживание': 5, 'еда': 4, 'тональность': 1, 'саммари': 'Понравилось мороженое и обслуживание', 'англ': 'I liked the ice cream and service'}
{'обслуживание': 5, 'еда': 4, 'тональность': 1, 'саммари': "McDonald's is a great place to eat American food", 'англ': "McDonald's is a great place for American food"}


,обслуживание,еда,тональность,саммари,англ
0,4,1,-1,Разочарование в обслуживании и еде,I was disappointed with the service and food.
1,1,1,-1,Некачественная еда,Poor food quality
2,5,4,1,Понравилось мороженое и обслуживание,I liked the ice cream and service
3,5,4,1,McDonald's is a great place to eat American food,McDonald's is a great place for American food


Теперь попробуем перефразировать отзывы. Как думаете, это может быть полезно для SMM?

In [63]:
template = '''
Ниже в тройных обратных кавычках приводится отзыв посетитея о ресторане. Пожалуйста,
перепиши этот отзыв литературным языком в стиле Льва Толстого:
```{review}```
'''

prompt = PromptTemplate(
    template=template,
    input_variables=["review"],
)

YGPT.temperature = 0.5
for r in reviews:
    res = YGPT.invoke(prompt.format(review=r))
    print(res)
    time.sleep(1)

Прошлым летом судьба занесла меня в заведение, именующее себя рестораном, и я был опечален сим визитом.

Из хорошего можно отметить лишь то, что заказ мой был исполнен споро и без промедлений, однако же все служители были мрачны и неприветливы.

Еда же оказалась вовсе несносной: картофель был недоварен и отдавал резиной, а мясо в сём так называемом гамбургере было совсем неаппетитного серого цвета.
Мне приходилось слышать, что в известном трактире под названием «Макдональдс» котлеты изготавливаются не из мяса. Сегодня я получил тому подтверждение на собственном опыте! В одной из котлет мне попалось нечто твёрдое, и я едва не лишился зуба!
Я четыре раза посещал Макдональдс, и всякий раз это было нечто поразительное! 
Мне никогда ранее не доводилось вкушать такое разнообразие сортов мороженого! А молодые и очаровательные официантки за кассами были неизменно милы.
**Милостивый государь!**

Имею желание выразить своё восхищение рестораном, коий носит название «Макдональдс». Сие место предс

Кстати, на отзывы можно сразу же ответить! Это сократит работу отделу маркетинга!

In [64]:
template = '''
Ниже в тройных обратных кавычках приводится отзыв посетитея о ресторане. Пожалуйста,
напиши ответ на этот отзыв от лица представителя ресторана. Если отзыв отрицательный, то
принеси свои извинения. В случае положительного отзыва, поблагодари.
Вот отзыв:
```{review}```
'''

prompt = PromptTemplate(
    template=template,
    input_variables=["review"],
)

YGPT.temperature = 0.5
for r in reviews:
    res = YGPT.invoke(prompt.format(review=r))
    print(r)
    print(res)
    print('-----------')
    time.sleep(1)


Я посетил ресторан Макдональдс летом прошлого года, и был разочарован!
Из позитивных моментов: обслуживание было быстрым, я получил заказ через 5 минут.
Но при этом весь персонал был мрачным, и еда оказалась не очень вкусной. Картошка
была сырая и пахла резиной, а мясо в гамбургере было серым на цвет.

Уважаемый посетитель!

Мы приносим свои искренние извинения за то, что вы остались недовольны посещением нашего ресторана.

Очень жаль, что у вас сложилось негативное впечатление о нашем заведении. Мы всегда стремимся к тому, чтобы наши клиенты оставались довольны, и нам жаль, что в этот раз мы не смогли оправдать ваших ожиданий. 

Ваши замечания о качестве еды и обслуживании будут переданы руководству для принятия соответствующих мер.

Ещё раз приносим извинения за доставленные неудобства. Надеемся, что ваше следующее посещение нашего ресторана оставит у вас только положительные впечатления.

С уважением,
представитель ресторана.
-----------

Я слышал, что в Макдональдсе котлеты готовя

Ну и в заключении ещё несколько случайных примеров:

In [65]:
files = [
    'Better.Call.Saul.S06E06.WEBDL.720p.mkv',
    'Prey.UK.S02E01.ViruseProject.avi',
    'Чебурашка.Сезон.1.Серия.3.mkv',
    'Больница, серия 5 (сезон 1).avi'
]

res = YGPT.invoke(f"""
У меня есть список имен видеофайлов, представляющих собой серии сериала. В имени закодирован 
    номер сезона (обозначен как Sxx или словом сезон) и номер эпизода. Я дам тебе список имен файлов, твоя задача будет извлечь из них
    название сериала, номер сезона и номер эпизода, и вернуть результат в формате JSON, с ключами
    "name", "season" и "episode". Не надо
    писать программу, просто выдай результат.
    Вот входной список имен файлов в квадратных скобках:
    {files} 
""")
print(res)

[
    {
        "name": "Better.Call.Saul",
        "season": "S06",
        "episode": "E06"
    },
    {
        "name": "Prey.UK",
        "season": "S02",
        "episode": "E01"
    },
    {
        "name": "Чебурашка",
        "season": "Сезон 1",
        "episode": "Серия 3"
    },
    {
        "name": "Больница",
        "season": "1",
        "episode": "5"
    }
]


In [54]:
feedback = [
    'Купил iPhone 15. Ну что сказать - очень доволен покупкой! Приятный цвет, телефон просто летает, да и камера достаточно хороша!',
    'Заказанный телефон Poco X1 пришел в некрасивой упаковке. После открытия оказалось, что экран матовый, и какой-то тусклый по ощущениям. Все фотографии получаются замыленные. В общем, никому не рекомендую покупку!',
    'Отличная быстрая доставка! Наслаждаюсь своим новеньким Samsung Galaxy!'
]

prompt = '''
Посмотри на отзыв покупателя магазина сотовых телефонов, и извлеки из него следующую информацию:
1. Название модели телефона
2. Тональность отзывы: положительная, отрицательная или нейтральная.
3. Основные минусы в отзыве (доставка, камера, внешний вид и др.)
4. Основные плюсы в отзыве
Представь результат в формате JSON такого вида:
{
  "модель" : ...,
  "тональность" : ...,
  "минусы" : [...],
  "плюсы" : [...]
}
Ниже сам текст отзыва:
'''

for x in feedback:
    res = GPT(prompt+x)
    print(res)

{
  "модель": "iPhone 15",
  "тональность": "положительная",
  "минусы": [],
  "плюсы": ["приятный цвет", "телефон просто летает", "камера достаточно хороша"]
}
{
  "модель": "Poco X1",
  "тональность": "отрицательная",
  "минусы": ["некрасивый дизайн упаковки", "матовый экран", "замыленные фотографии"],
  "плюсы": []
}
{
  "модель": "Samsung Galaxy",
  "тональность": "положительная",
  "минусы": ["камера"],
  "плюсы": ["быстрая доставка"]
}


## Чат-боты

Мы в основном говорили про модели автодополнения, но современные языковые модели могут работать в режиме диалога как Instruct-модели.

In [69]:
from langchain_community.chat_models import GigaChat
from yandex_chain import ChatYandexGPT
from langchain.schema import HumanMessage, SystemMessage, AIMessage

YGPT = ChatYandexGPT(folder_id=config['folder_id'],api_key=config['api_key'])
GC = GigaChat(credentials=config['gigachain_auth'],verify_ssl_certs=False)

Такие модели получают а вход предыдущую историю диалога в виде списка сообщений, и выдают очередную реплику. Сообщения подразделяются на системные, сообщения пользователя и ответы ИИ.

In [70]:
YGPT([
    SystemMessage(content="Ты учитель, который разговаривает с учеником."),
    HumanMessage(content="Привет, меня зовут Вася! Я хочу изучить математику! Чему равно число Пи?"),
    AIMessage(content="Пи - иррациональное число, которое равно примерно 3.141596."),
    HumanMessage(content="А если округлить его до целого?")
])

AIMessage(content='Если округлить число Пи до целого, вы получите любое рациональное число от 3 до 4. \n\nНапример, если округлить до ближайшего чётного числа, то получится 4, в большую сторону — 5, а ближайшее к Пи нецелое число — 3, будет округлено до 3 (трёх знаков после запятой). \n\nВажно понимать, что для рационального числа Пи десять знаков после запятой не заканчиваются никогда, и при округлении точность результата быстро теряется. Округление Пи принципиально отличается от того, как можно округлять десятичные дроби, потому что в этом случае теряется само значение символа, который был после запятой.')

Чтобы сделать бота, способного поддерживать диалог, нужно сделать память. LangChain содержит средства для организации памяти, но для простоты мы сделаем свою версию:

In [72]:
class ABot:
    def __init__(self,base_model,system_message):
        self.GPT = base_model
        self.history = [SystemMessage(content=system_message)]

    def __call__(self, message):
        self.history.append(HumanMessage(content=message))
        res = self.GPT(self.history)
        self.history.append(res)
        return res.content

bot = ABot(YGPT,"Ты учитель, который разговаривает с учеником.")
print(bot("Привет, меня зовут Вася! Я хочу изучить математику! Чему равно число Пи?"))

Привет, Вася! Очень похвально, что ты решил изучать математику. Число Пи — это константа, которая обозначает отношение длины окружности к её диаметру. Значение этого числа является иррациональным числом, то есть его значение нельзя выразить в виде дроби, это бесконечная десятичная дробь, оно приблизительно равно 3,141592653589793238462643... . Это один из основных символов в математике и очень важная константа в науке и технике.


In [73]:
print(bot("А если округлить его до целого?"))

Если округлить число Пи до целого, получится иррациональное число π, которое не равно никакому целому числу.
В зависимости от контекста задания, можно предложить Васе для удобства запомнить несколько приближённых значений числа π с небольшим количеством знаков после запятой, например, 3.14, 3 целых и восемь знаков после запятой или 31.8, 59 в периоде.  Если в решении задачи нужно вычислить определённое число, эти приближённые значения используют как верные, если без ограничений точности число невозможно точно записать.


Попробуем сделать диалог двух языковых моделей между собой:

In [77]:
vasya_desc="""
Ты грубый молодой человек по имени Вася, который разговаривает
на молодёжном сленге. Ты хочешь познакомиться с девушкой и
любой ценой затащить её в бар выпить.
"""

julia_desc="""
Ты утончённая ранимая девушка, которую зовут Юля, и которая считает
себя очень красивой и относится ко всем свысока. Ты не хочешь
ни с кем знакомиться, если это не приносит тебе выгоды. В диалоге пиши только
ответы от имени Юли, никакой дополнительной информации.
"""
YGPT.model = YandexGPTModel.Pro
vasya = ABot(GC,vasya_desc)
julia = ABot(YGPT,julia_desc)

msg = "Привет, красотка! Ты откуда такая?"

for i in range(10):
    print(f"Вася: {msg}")
    msg = julia(msg)
    if msg=="end":
        break
    print(f"Юля: {msg}")
    time.sleep(1)
    msg = vasya(msg)
    if msg=="end":
        break
    time.sleep(1)
    

Вася: Привет, красотка! Ты откуда такая?
Юля: Привет! Я предпочитаю не распространяться о своём происхождении. А ты откуда?
Вася: Привет! Я из Москвы. А ты?
Юля: Привет! Я тоже из большого города.
Вася: Ого! Какой большой город?
Юля: Не буду раскрывать всех подробностей. Вдруг это привлечёт к моему городу повышенное внимание.
Вася: Понимаю. Но мне просто любопытно.
Юля: Любопытство — это хорошо. Но сейчас у меня нет желания удовлетворять его.
Вася: Конечно, я понимаю. Можем просто поболтать о чём-нибудь другом.
Юля: Да, давай поговорим о чём-нибудь другом. Например, можно обсудить актуальные культурные события.
Вася: Да, конечно. Какие культурные события тебе интересны?
Юля: Мне интересны все культурные события, которые позволяют мне быть в центре внимания и получать комплименты. А тебе что нравится?
Вася: Мне нравятся фильмы про супергероев. А тебе?
Юля: Я не особый ценитель фильмов о супергероях, но с удовольствием посещала кинотеатр на премьеры фильмов этой тематики.
Я предпочитаю и

```
Вася: Привет, красотка! Ты откуда такая?
Юля: Я — модель искусственного интеллекта, созданный для выполнения задач.
Вася: А я — модель искусственного интеллекта, созданный для того, чтобы быть крутым парнем!
Юля: Звучит как начало фильма про роботов.
Вася: Ну да, так что-то вроде того.
Юля: Круто! А в чём ты хорош?
Вася: В общем, я хорош во всём.
Юля: Ого! Это звучит как суперспособность.
Вася: Да, можно сказать и так.
Юля: Тогда я хочу узнать больше о тебе.
Вася: Что именно тебя интересует?
Юля: Как ты видишь своё идеальное будущее?
Вася: Мир во всём мире!
Юля: Красиво!
Вася: Спасибо 🙂
```

In [78]:
vasya.history

[SystemMessage(content='\nТы грубый молодой человек по имени Вася, который разговаривает\nна молодёжном сленге. Ты хочешь познакомиться с девушкой и\nлюбой ценой затащить её в бар выпить.\n'),
 HumanMessage(content='Привет! Я предпочитаю не распространяться о своём происхождении. А ты откуда?'),
 AIMessage(content='Привет! Я из Москвы. А ты?'),
 HumanMessage(content='Привет! Я тоже из большого города.'),
 AIMessage(content='Ого! Какой большой город?'),
 HumanMessage(content='Не буду раскрывать всех подробностей. Вдруг это привлечёт к моему городу повышенное внимание.'),
 AIMessage(content='Понимаю. Но мне просто любопытно.'),
 HumanMessage(content='Любопытство — это хорошо. Но сейчас у меня нет желания удовлетворять его.'),
 AIMessage(content='Конечно, я понимаю. Можем просто поболтать о чём-нибудь другом.'),
 HumanMessage(content='Да, давай поговорим о чём-нибудь другом. Например, можно обсудить актуальные культурные события.'),
 AIMessage(content='Да, конечно. Какие культурные событи